# **Psychology-6k Semantic Retrieval :Il match semantico delle risposte in psicologia**

Il progetto sviluppato realizza un **sistema di retrieval semantico** capace di associare un input (domanda) all’output corretto (risposta) all’interno del dataset Psychology-6k, disponibile su HuggingFace.

Il flusso è organizzato come pipeline:

* **Preparazione e pulizia dei testi** – sono state testate due modalità: una light (normalizzazione di base) e una con lemmatizzazione tramite SpaCy. Quest’ultima non era strettamente necessaria per il progetto, ma è stata integrata a fini didattici.

* **Trasformazione in embeddings** – tramite il modello thenlper/gte-small di SentenceTransformers.

* **Calcolo della similarità coseno** – per stimare la vicinanza semantica tra domande e risposte candidate.

* **Ranking e valutazione** – con la metrica Mean Reciprocal Rank (MRR), che misura la posizione media della risposta corretta nella classifica.

* **Interfaccia interattiva (Gradio)** – per esplorare le domande, confrontare le due pipeline e visualizzare i risultati.

*Risultati*
Il sistema ha ottenuto valori di **MRR molto elevati: ≈0.97** con la pipeline light e ≈0.96 con la pipeline lemma, confermando che la risposta corretta viene classificata quasi sempre al primo posto.

##Caricamento Dataset
Ho caricato il dataset **Psychology-6k** direttamente da HuggingFace (`samhog/psychology-6k`).

**Dettagli del dataset:**
- **Modalities:** Text  
- **Format:** JSON  
- **Size:** 1K – 10K  
- **Librerie utilizzabili:** 🤗 Datasets, 🐼 Pandas

In questa fase verifico la struttura dei dati e preparerò il DataFrame per l’analisi.


In [ ]:
#Caricamento dataset da HuggingFace
from datasets import load_dataset

# Carico il dataset direttamente da HuggingFace
dataset = load_dataset("samhog/psychology-6k")

# Converto la parte 'train' in DataFrame Pandas per analisi
import pandas as pd
df = dataset["train"].to_pandas()

In [ ]:
# Decido di mostrare alcune righe del Dataset
df.head(3)

In [ ]:
# Qui mostro, per le prime tre righe del dataset, la coppia domanda (input)
# e la relativa risposta corretta (gold answer)
for i in range(3):
    print(f"Domanda: {df['input'][i]}")
    print(f"Risposta gold: {df['output'][i]}")
    print("-----")

## EDA: Analisi preliminare Esplorativa del Dataset Psychology- 6K

In [ ]:
# EDA: Analisi Preliminare

# Conteggio di record totali
print("Numero di record totali:", len(df))

#print("Numero di colonne totali:", len(df.columns))
#print(df.columns)


#Verifico la struttara del dataset se ha split (train/valid/test)
print("Split presenti:", dataset.keys())  # es. ['train'] oppure ['train','test']

#Valori mancanti (NaN o stringhe vuote)
print("\nValori mancanti per colonna:")
print(df.isna().sum())
print("\nEsempi di domande vuote:", df[df["input"].isna()])

# Duplicati
dup_questions = df.duplicated(subset=["input"]).sum()
print("\nDomande duplicate:", dup_questions)

# Trova tutte le righe duplicate (identiche su tutte le colonne)
print(df[df.duplicated()].head(10))

dup_q_and_ans = df.duplicated(subset=["input", "output"]).sum()
print("Coppie domanda-risposta duplicate:", dup_q_and_ans)

# Distribuzione lunghezze (in caratteri)
df["q_len_char"] = df["input"].astype(str).str.len()
print("\nStatistiche lunghezze domande (caratteri):")
print(df["q_len_char"].describe())

# Distribuzione lunghezze (in token/parole)
df["q_len_tokens"] = df["input"].astype(str).str.split().apply(len)
print("\nStatistiche lunghezze domande (token):")
print(df["q_len_tokens"].describe())

# Visualizzo un campione di domande molto corte o molto lunghe
print("\nDomande corte:")
print(df[df["q_len_tokens"] < 5]["input"].head(3))
print("\nDomande lunghe:")
print(df[df["q_len_tokens"] > 30]["input"].head(3))

Conclusione dell' EDA

L’analisi esplorativa del dataset *Psychology-6k* ha mostrato che:

- **Dimensione**: 5846 record, tutti nell'unico split presente `train.
- **Qualità dei dati**: nessun valore mancante o domanda vuota, dati quindi completi.  
- **Duplicati**: circa 1489 domande duplicate (25%). Questo indica che un quarto delle domande si ripete, probabilmente per coprire più formulazioni di casi simili. Tale ridondanza potrebbe influenzare la fase di retrieval, semplificando eccessivamente la ricerca della risposta. Rilevando dei duplicati o notato che non  ci sono duplicati esatti, ma alcune domande compaiono con risposte diverse. Non lo considero un errore, bensì un limite del dataset: introduce ambiguità che possono influenzare la valutazione con MRR.
- **Distribuzione lunghezze**: domande di lunghezza media (≈13 token), con minimo 4 e massimo 43 token. La variabilità è contenuta, con poche domande molto lunghe che descrivono situazioni complesse (es. depressione, ansia, dipendenze).  
- **Struttura**: il dataset non fornisce direttamente una lista di **risposte candidate**. Sono presenti solo la **domanda** (`input`) e la **risposta corretta** (`output`). Per la fase di retrieval sarà quindi necessario costruire un set di risposte candidate, ad esempio campionando da altre risposte del dataset.

**Conclusione:**  
Il dataset presenta una buona qualità complessiva: è completo, ben bilanciato nelle lunghezze e senza valori nulli. Gli aspetti critici principali sono:  
1. la presenza di domande duplicate (≈25%), che richiede attenzione in fase di valutazione;  
2. l’assenza di candidate answers predefinite, che dovranno essere generate artificialmente per testare il sistema di retrieval.



## **Pulizia e normalizzazione del testo**


**Tecniche di pre-processing utilizzate**

In questa fase ho applicato due tipi di pre-processing:

1. **Pulizia leggera**  
   Ho normalizzato il testo portandolo in minuscolo, eliminando caratteri rumorosi (emoji e simboli non alfanumerici) e compattando gli spazi.  
   Questa scelta serve a rendere i dati più uniformi senza alterare troppo il contesto, cosa utile per modelli di tipo SBERT che beneficiano di frasi naturali.

2. **Lemmatizzazione con SpaCy + filtro POS**  
   Ho usato SpaCy per ridurre le parole al loro lemma (es. *running*, *ran* → *run*) e ho mantenuto solo le parole di contenuto (sostantivi, verbi, aggettivi e nomi propri).  
   In questo modo riduco la variabilità morfologica e mi concentro sugli elementi più significativi delle frasi.

Ho quindi applicato **due tecniche linguistiche** (lemmatizzazione e POS tagging).



In [ ]:
# Pulizia e normalizzazione del testo
#importo la libreria

import re
import spacy

# In questa parte definisco due modalità di pulizia:
# - pulizia leggera: normalizza il testo senza stravolgerlo
# - lemmatizzazione: riduce le parole alla radice, così riduco la variabilità morfologica


# Qui creo una whitelist di caratteri che voglio mantenere:
_NOISE_CHARS = r"[^a-zA-Z0-9\s\.\,\?\!\:\;\-\(\)\'\"]"

def clean_text_light(text: str) -> str:

    #Pulizia leggera del testo:
    # - converto tutto in minuscolo
    #- tolgo caratteri strani (emoji, simboli non utili)
    #- compatto gli spazi
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(_NOISE_CHARS, " ", text)   # tolgo caratteri non ammessi
    text = re.sub(r"\s+", " ", text).strip() # elimino spazi multipli
    return text

# Applico la pulizia leggera alle colonne input e output
df["input_clean"]  = df["input"].apply(clean_text_light)
df["output_clean"] = df["output"].apply(clean_text_light)



# LEMMATIZZAZIONE

# In questa parte uso SpaCy per portare le parole al loro lemma

# Carico il modello di SpaCy
try:
    nlp = spacy.load("en_core_web_sm", disable=["parser"])
except OSError:
    nlp = spacy.load("en_core_web_sm", disable=["parser"])

KEEP_POS = {"NOUN", "PROPN", "VERB", "ADJ"}  # i POS che considero utili

def lemmatize_spacy(text: str, remove_stop: bool = True) -> str:

    #Lemmatizzazione con filtro POS:
    #- mantengo solo le parole di contenuto
    #- tolgo le stopwords

    if not isinstance(text, str):
        return ""
    doc = nlp(text)
    tokens = []
    for tok in doc:
        if remove_stop and tok.is_stop:
            continue
        if tok.pos_ not in KEEP_POS:
            continue
        lemma = tok.lemma_.strip()
        if lemma:
            tokens.append(lemma)
    return " ".join(tokens)

# Creo anche le colonne con la versione lemmatizzata
df["input_lemma"]  = df["input_clean"].apply(lemmatize_spacy)
df["output_lemma"] = df["output_clean"].apply(lemmatize_spacy)



# Qui confronto le tre versioni: testo originale, pulito e lemmatizzato.
df[["input", "input_clean", "input_lemma", "output", "output_clean", "output_lemma"]].head(3)


### Candidate Answers

Nella fase successiva costruisco i **candidate answers** per ogni domanda.  
Il dataset *Psychology-6k* infatti fornisce solo la domanda (`input`) e la risposta corretta (`output`), ma non una lista di alternative.  

Per poter testare il retrieval, genero quindi per ogni domanda:  
- 1 risposta corretta (gold)  
- 3 risposte errate scelte casualmente dal resto del dataset  

Questa operazione la ripeto **per entrambe le versioni del dataset** (light e lemma), in modo da poter confrontare le prestazioni del sistema in condizioni equivalenti ma con pre-processing diverso.


In [ ]:
#  Costruzione dei Candidate Answers
# In questa fase preparo, per ogni domanda, un set di risposte:
# - 1 risposta corretta (gold)
# - alcune risposte sbagliate prese a caso dal dataset (negatives)
# Questo è necessario perché nel dataset Psychology-6k non ci sono già candidate answers.

import random

# imposto un seme random fisso così i risultati sono riproducibili
random.seed(42)

def build_candidates(df, question_col, answer_col, k_neg=3):

  #  Per ogni domanda creo un dizionario con:
  # - question: la domanda
  # - gold: la risposta corretta
  # - candidates: lista con 1 gold + k_neg risposte errate prese a caso dal
  # dataset

    all_answers = df[answer_col].tolist()
    N = len(df)
    records = []

    for i, row in df.iterrows():
        q = row[question_col]
        gold = row[answer_col]

        # seleziono k_neg risposte errate diverse dalla gold
        negatives = []
        while len(negatives) < k_neg:
            j = random.randrange(N)
            cand = all_answers[j]
            if cand != gold and cand not in negatives:
                negatives.append(cand)

        candidates = [gold] + negatives
        random.shuffle(candidates)  # mescolo così la gold non è sempre in prima posizione

        records.append({
            "question": q,
            "gold": gold,
            "candidates": candidates
        })
    return records



In [ ]:
records = build_candidates(df, "input", "output", k_neg=3)

# mostro i primi 3 esempi costruiti
for r in records[:3]:
    print("Domanda:", r["question"])
    print("Risposta gold:", r["gold"])
    print("Candidates:", *r["candidates"], sep="\n - ")
    print("------------")

### Nota: **confronto**

Dopo aver pulito e normalizzato il dataset, ho deciso di preparare **due versioni parallele** dei dati:  

1. **Pulizia leggera (light):** testo convertito in minuscole, rimozione del rumore e gestione spazi, mantenendo la punteggiatura utile.  
2. **Pulizia + Lemmatizzazione (lemma):** stessa pulizia leggera, ma in più riduzione delle parole alla loro radice tramite SpaCy (es. *running*, *ran*, *runs* → *run*).  

L’obiettivo è confrontare questi due approcci e valutare se la lemmatizzazione porta effettivamente un miglioramento nelle prestazioni del retrieval rispetto alla semplice pulizia leggera.


In [ ]:
# Qui preparo i dati per le due modalità:
# - pulizia leggera (light)
# - pulizia + lemmatizzazione (lemma)

# Per la pulizia leggera uso input_clean e output_clean
records_light = build_candidates(df, question_col="input_clean", answer_col="output_clean", k_neg=3)

# Per la pulizia con lemma uso input_lemma e output_lemma
records_lemma = build_candidates(df, question_col="input_lemma", answer_col="output_lemma", k_neg=3)

# Controllo un esempio per ogni modalità
print("Esempio (pulizia leggera):")
print(records_light[0])

print("\nEsempio (pulizia + lemma):")
print(records_lemma[0])

## **Embedding**

### Generazione Embeddings (light vs lemma)

In questa fase trasformo testi (domande e risposte candidate) in vettori numerici usando il modello `thenlper/gte-small`.
Creo due insiemi separati di embeddings:
- **light** → ottenuti dai testi puliti “leggeri”,
- **lemma** → ottenuti dai testi puliti + lemmatizzati.

Scelgo di **deduplicare** i testi prima di codificarli (question + tutti i candidates) per velocizzare e risparmiare memoria.
Normalizzo gli embeddings così che il **prodotto scalare = similarità coseno**, utile per la fase di retrieval.
Ho configurato batch_size di 60

In [ ]:
#  Embeddings con SentenceTransformers (light vs lemma)

# qui importo le librerie necessarie per gli embeddings
#Embeddings con SentenceTransformers: versione "pulita" senza cache globale

import numpy as np
from sentence_transformers import SentenceTransformer
import pickle

# scelgo il modello con cui lavorare
MODEL_NAME = "thenlper/gte-small"

def make_model(model_name: str = MODEL_NAME):

    #Creo l'istanza del modello una sola volta e la ritorno.
    #Preferisco farlo qui (helper) così ho il punto unico dove eventualmente
    #configurare device/dtype/log ecc.

    return SentenceTransformer(model_name)

def encode_texts(texts, model: SentenceTransformer, batch_size: int = 60, normalize: bool = True):

  #Trasformo una lista di testi in embeddings usando *l'istanza* del modello
  #che passo esplicitamente (niente stato globale).
  #Se normalize=True, gli embedding sono L2-normalizzati e il dot product = #coseno.

    return model.encode(
        texts,
        batch_size=batch_size,
        convert_to_numpy=True,
        show_progress_bar=True,
        normalize_embeddings=normalize
    )

def build_embedding_store(records, model: SentenceTransformer, batch_size: int = 60):

  #Costruisco un dizionario {testo -> embedding} per il set di record passato.
  #- raccolgo tutte le stringhe (domande + candidati)
  #- deduplica per evitare ricalcoli inutili
  #- codifico in batch con l'istanza del modello

    uniq_texts = set()
    for rec in records:
        uniq_texts.add(rec["question"])
        for cand in rec["candidates"]:
            uniq_texts.add(cand)

    uniq_texts = list(uniq_texts)
    embs = encode_texts(uniq_texts, model=model, batch_size=batch_size, normalize=True)

    store = {txt: vec for txt, vec in zip(uniq_texts, embs)}
    return store

def store_stats(store, label: str):

    #Stampo due numeri per capire 'quanto pesa' lo store:
    #- quanti vettori
    #- dimensione del vettore
    #- stima memoria (float32 ~ 4 byte)

    n_vec = len(store)
    dim = next(iter(store.values())).shape[0] if n_vec > 0 else 0
    approx_mb = n_vec * dim * 4 / (1024**2)
    print(f"{label}: vettori={n_vec}, dim={dim}, ~mem={approx_mb:.1f} MB")

def quick_check(records, store, label: str):

    #Controllo veloce: per il primo record verifico che il prodotto scalare
    #(coseno se normalizzati) tiri fuori il candidato top.

    print(f"\nQuick check ({label})")
    rec = records[0]
    q = rec["question"]
    cands = rec["candidates"]

    qv = store[q]
    cv = np.stack([store[c] for c in cands], axis=0)

    scores = (cv @ qv).astype(float)
    top_idx = int(np.argmax(scores))

    print("Q    :", q)
    print("GOLD :", rec["gold"])
    print("TOP  :", cands[top_idx])
    print("Scores:", np.round(scores, 3))

#Punto di ingresso: istanzio il modello UNA volta e lo passo dove serve

model = make_model(MODEL_NAME)

print("→ Costruisco embedding store (light)…")
emb_store_light = build_embedding_store(records_light, model=model)

print("→ Costruisco embedding store (lemma)…")
emb_store_lemma = build_embedding_store(records_lemma, model=model)

# qualche statistica rapida
store_stats(emb_store_light, "Store LIGHT")
store_stats(emb_store_lemma, "Store LEMMA")

# mini check su un record per modalità
quick_check(records_light, emb_store_light, "light")
quick_check(records_lemma, emb_store_lemma, "lemma")

# salvo su disco per riuso in valutazione (evito ricalcoli)
with open("emb_store_light.pkl", "wb") as f:
    pickle.dump(emb_store_light, f)
with open("emb_store_lemma.pkl", "wb") as f:
    pickle.dump(emb_store_lemma, f)

print("\nEmbeddings pronti per retrieval + MRR ")



### Nota sul log di download e calcolo embeddings

All’inizio il modello thenlper/gte-small veniva caricato tramite la cache globale di SentenceTransformers: ogni funzione richiamava il modello implicitamente, con maggiore rischio di confusione e ricalcoli.

Ho poi cambiato piano, passando a una strategia più chiara:
* istanziare il modello una sola volta con make_model,
* passarlo esplicitamente alle funzioni che calcolano gli embeddings.

In questo modo evitiamo dipendenze globali, rendiamo il codice più trasparente e controlliamo meglio l’uso della memoria.

Subito dopo inizia il calcolo degli embeddings:

Ogni frase (domanda + risposte candidate) viene convertita in un vettore numerico.

Il log mostra l’avanzamento in batch: ad esempio 64/160 indica che sono stati processati 64 batch su un totale di 160.

Sono riportati anche percentuale di completamento, tempo trascorso e stima del tempo rimanente.

Queste informazioni sono utili per monitorare l’avanzamento del processo e stimare la durata della generazione degli embeddings sull’intero dataset.


##**Retrieval e Valutazione (MRR)**

### Retrieval + Valutazione (MRR)

In questa fase consumo gli embeddings calcolati nella precedente processo per:
1) confrontare ogni **domanda** con le sue **candidate answers** via similarità coseno,
2) calcolare la **Mean Reciprocal Rank (MRR)** per misurare quanto spesso la risposta corretta è in alto nel ranking.

Eseguo tutto **due volte**:
- **light** → usando gli embeddings generati dai testi con pulizia leggera,
- **lemma** → usando gli embeddings generati dai testi lemmatizzati.

Confronto i due MRR e mostro anche alcuni esempi “difficili” per analisi qualitativa.


In [ ]:
# Retrieval + MRR (confronto light vs lemma)

import numpy as np
import pandas as pd

# qui definisco una funzione che mi calcola la RR per una domanda
def reciprocal_rank(gold_idx: int, scores: np.ndarray) -> float:

    #RR = 1 / (posizione della risposta corretta nel ranking, 1-based).
    #'scores' sono le similarità (coseno) tra domanda e candidati.

    order = np.argsort(-scores)             # indici dei candidati ordinati per score decrescente
    rank_0_based = np.where(order == gold_idx)[0][0]  # posizione 0-based della gold
    return 1.0 / (rank_0_based + 1)         # converto a 1-based

# qui definisco una funzione che valuta un intero set (records + store embeddings)
def evaluate_mrr(records, emb_store, n_examples_to_log=3):

    #Scorro tutte le domande:
    #- prendo embedding della domanda e dei candidati dallo 'store'
    #- calcolo similarità (dot product perché già normalizzati)
    #- accumulo le RR per mediare in MRR
    #- raccolgo qualche dettaglio qualitativo (casi difficili)

    rr_list = []
    details = []

    for rec in records:
        q = rec["question"]
        cands = rec["candidates"]
        gold = rec["gold"]
        gold_idx = cands.index(gold)

        # recupero i vettori dal magazzino (store) usando il testo come chiave
        qv = emb_store[q]                              # shape: (d,)
        cv = np.stack([emb_store[c] for c in cands])   # shape: (k, d)

        # prodotto scalare = similarità coseno
        scores = (cv @ qv).astype(float)               # shape: (k,)

        rr = reciprocal_rank(gold_idx, scores)
        rr_list.append(rr)

        # salvo un po' di info sul caso (mi serve per stampare esempi)
        top_idx = int(np.argmax(scores))
        details.append({
            "question": q,
            "gold": gold,
            "top_pred": cands[top_idx],
            "rr": rr,
            "rank_gold": int(1/rr)
        })

    mrr = float(np.mean(rr_list))
    # ordino i dettagli per mettere in alto i casi peggiori (rank_gold > 1)
    details_sorted = sorted(details, key=lambda d: d["rank_gold"], reverse=True)
    return mrr, details_sorted[:n_examples_to_log]

# qui eseguo la valutazione per entrambe le modalità (light e lemma)
results = {}

print(" Valuto MRR (light)…")
mrr_light, hard_light = evaluate_mrr(records_light, emb_store_light, n_examples_to_log=3)
results["light"] = {"MRR": mrr_light, "hard": hard_light}

print(" Valuto MRR (lemma)…")
mrr_lemma, hard_lemma = evaluate_mrr(records_lemma, emb_store_lemma, n_examples_to_log=3)
results["lemma"] = {"MRR": mrr_lemma, "hard": hard_lemma}

# tabella riassuntiva dei due MRR
summary = pd.DataFrame([
    {"mode": "light", "MRR": results["light"]["MRR"]},
    {"mode": "lemma", "MRR": results["lemma"]["MRR"]}
]).sort_values("MRR", ascending=False)

print("\n Confronto MRR")
display(summary)

# stampo alcuni esempi “difficili” per ciascuna modalità (gold non al rank 1)
def print_hard_examples(mode_label, hard_list):
    print(f"\nEsempi difficili – {mode_label}")
    if not hard_list:
        print("(Nessun esempio difficile nei primi 3 casi richiesti)")
        return
    for ex in hard_list:
        print("-" * 70)
        print("Q   :", ex["question"])
        print("GOLD:", ex["gold"])
        print("TOP :", ex["top_pred"])
        print("RR  :", round(ex["rr"], 3), f"(rank_gold = {ex['rank_gold']})")

print_hard_examples("light", results["light"]["hard"])
print_hard_examples("lemma", results["lemma"]["hard"])


###**Conclusioni e confronto Light vs Lemma**

Dopo aver confrontato le due pipeline di pre-processing, i risultati mostrano che:

* Pulizia leggera (light): ottiene MRR = 0.975

* Pulizia + Lemmatizzazione (lemma): ottiene MRR = 0.958

In entrambi i casi il ranking è quasi sempre corretto al primo posto. I valori sono molto alti e indicano un sistema di retrieval efficace, con un leggero vantaggio per la modalità light.

La spiegazione sta nel fatto che modelli come gte-small (SentenceTransformer) sono addestrati su frasi naturali: conservare la forma originale (con punteggiatura e variazioni morfologiche) aiuta il modello a sfruttare meglio il contesto semantico.
Con la lemmatizzazione, invece, le frasi risultano più astratte e meno ricche di informazioni, riducendo la capacità del modello di distinguere le sfumature di significato.

**Limiti del dataset**

Come discusso all’inizio, il dataset Psychology-6k presenta alcune domande duplicate o molto simili, con risposte che variano per coprire più casi. Questo aspetto ha probabilmente influenzato la valutazione della metrica MRR: in presenza di risposte quasi identiche, il sistema può ritrovare facilmente la gold answer o una sua variante, “gonfiando” leggermente il punteggio. In altre parole, la metrica può risultare ottimistica perché non penalizza la ridondanza, restituendo un’immagine più rosea delle reali capacità del sistema.

**Scelta del modello**

Avremmo potuto sperimentare con altri SentenceTransformer, ad esempio all-mpnet-base-v2 o bge-small-en-v1.5. Questi modelli, più grandi o più recenti, avrebbero potuto ottenere prestazioni migliori grazie a un embedding più ricco, ma con un costo maggiore in termini di memoria e tempo di calcolo.

**Scelta della Metrica**

Oltre alla Mean Reciprocal Rank (MRR), avremmo potuto utilizzare la Mean Average Precision (MAP).

* MRR valuta solo la posizione della prima risposta corretta nel ranking. Se la gold answer è in prima posizione, il punteggio è 1; se è in terza, vale 1/3, e così via. Questo rende la metrica molto sensibile al primo match trovato.

* MAP, invece, calcola la precisione media lungo tutto il ranking e la fa sulla media delle query. È quindi più adatta quando una domanda ha più risposte corrette, perché misura non solo se trovi la prima risposta giusta, ma anche come distribuisci tutte le risposte giuste nel ranking.

Nel nostro dataset ogni domanda ha una sola gold answer, quindi la MRR appare adeguata. Tuttavia, la presenza di domande duplicate o molto simili con risposte leggermente diverse ha influito sul risultato:

* Con la MRR, il sistema ottiene punteggi alti perché basta che una delle risposte quasi identiche finisca in alto per avere un reciproco vicino a 1.

* Con la MAP, invece, questa ridondanza non avrebbe gonfiato i valori: la metrica avrebbe considerato l’intero ordinamento e avrebbe dato un peso più equilibrato alla presenza di varianti multiple della stessa risposta.

Quindi:
* La MRR è semplice, immediata e sufficiente in presenza di una sola gold answer.

* La MAP sarebbe stata più robusta nel gestire i duplicati, ma al costo di maggiore complessità e senza un reale beneficio nel nostro dataset specifico.

#Estensioni




## GUI Retrieval con Gradio

In [ ]:
# ================================================================
# GUI Retrieval con Gradio (light vs lemma)
# ---------------------------------------------------------------
# Obiettivo dell'interfaccia:
# - seleziono il pre-processing ("light" o "lemma")
# - inserisco una domanda e scelgo Top-k
# - vedo le migliori risposte dal pool (derivato dal dataset)
#
# Nota architetturale:

# - Gli embeddings del pool vengono pre-calcolati una volta usando la stessa istanza.
# - In GUI usiamo `gr.State` per condividere oggetti tra callback
#
# Prerequisiti:
# - Variabili già disponibili: `model`, `df`, `clean_text_light`, `lemmatize_spacy`
# - Colonne dataset già pronte: df['output_clean'], df['output_lemma']
# ================================================================

import numpy as np
import pandas as pd

try:
    import gradio as gr
except ImportError:
    # !pip -q install gradio
    import gradio as gr

# ---------------------------
# Utility: encoding normalizzato
# ---------------------------
def encode_norm(texts, model, batch_size=128):
    """Genero embeddings normalizzati (così dot product = cosine)."""
    return model.encode(
        texts,
        batch_size=batch_size,
        convert_to_numpy=True,
        show_progress_bar=False,
        normalize_embeddings=True
    )

# ------------------------------------------------------
# Preparo i pool di risposte (light/lemma) dal DataFrame
# - deduplica per velocizzare ed evitare bias di duplicati
# ------------------------------------------------------
answer_pool = {
    "light": sorted(set(df["output_clean"].dropna().astype(str).tolist())),
    "lemma": sorted(set(df["output_lemma"].dropna().astype(str).tolist())),
}

# ------------------------------------------------------
# Pre-calcolo embeddings dei pool con la STESSA istanza `model`
# - Questo permette query-time veloce (si embedda solo la query)
# ------------------------------------------------------
pool_embs = {}
for mode in ["light", "lemma"]:
    print(f"↻ Pre-calcolo embeddings del pool risposte [{mode}] …")
    pool_embs[mode] = encode_norm(answer_pool[mode], model=model)

# -----------------------------------------
# Pre-processing della query coerente al mode
# -----------------------------------------
def preprocess_query(q: str, mode: str) -> str:
    """Applico lo stesso pre-processing della pipeline alla query."""
    if not q or not isinstance(q, str):
        return ""
    if mode == "lemma":
        # prima pulizia leggera, poi lemma (come nella pipeline)
        q_clean = clean_text_light(q)
        q_lemma = lemmatize_spacy(q_clean, remove_stop=True)
        return q_lemma
    # modalità light
    return clean_text_light(q)

# -----------------------------------------
# Funzione principale di retrieval (callback GUI)
# -----------------------------------------
def retrieve(query: str, mode: str, topk: int, model, pool_embs, answer_pool):
    """
    - Applico pre-processing coerente
    - Embedding della query con la STESSA istanza `model`
    - Similarità coseno con il pool (già normalizzato)
    - Ritorno Top-k (risposta, score) + mini nota "perché"
    """
    if not query or not query.strip():
        return [], "Inserisci una domanda per avviare la ricerca."

    # Pre-process query
    q_proc = preprocess_query(query, mode)

    # Embedding query (shape: (d,))
    qv = encode_norm([q_proc], model=model)[0]

    # Pool selezionato e suoi embeddings
    cands = answer_pool[mode]
    cv = pool_embs[mode]  # shape: (N, d)

    # Similarità coseno = dot product (vettori normalizzati)
    scores = cv @ qv  # shape: (N,)

    # Ordinamento per punteggio decrescente e Top-k (limitato 1..10)
    topk = int(max(1, min(topk, 10)))
    order = np.argsort(-scores)[:topk]

    # Risultati formattati per tabella (risposta, score)
    results = [[cands[i], float(scores[i])] for i in order]

    # Nota esplicativa: margine Top1–Top2 (se disponibili)
    if len(order) >= 2:
        margin = float(scores[order[0]] - scores[order[1]])
        why = f"Margine top1–top2: {margin:.3f} (più è alto, più la scelta è netta)."
    else:
        why = "Mostro una sola risposta (Top-1)."

    return results, why

# ---------------------------
# Costruzione interfaccia GUI
# ---------------------------
with gr.Blocks() as demo:
    gr.Markdown("## Retrieval Q→A – Demo (light vs lemma)")
    gr.Markdown(
        "Seleziona il **pre-processing** (light/lemma), inserisci una **domanda** e scegli **Top-k**. "
        "La lista mostra le migliori risposte dal pool del dataset. Gli score sono similarità coseno."
    )

    with gr.Row():
        mode = gr.Radio(choices=["light", "lemma"], value="light", label="Pre-processing")
        topk = gr.Slider(1, 10, value=3, step=1, label="Top-k")

    query = gr.Textbox(label="Domanda (in inglese)")

    out_tbl = gr.Dataframe(headers=["Risposta", "Score"], datatype=["str", "number"], wrap=True)
    out_why = gr.Markdown()

    # Stati condivisi (no global): passo oggetti alla callback in modo esplicito
    # Wrap model in a lambda to prevent Gradio from calling it directly
    st_model = gr.State(lambda: model)
    st_pool_embs = gr.State(pool_embs)
    st_answer_pool = gr.State(answer_pool)

    btn = gr.Button("Cerca")
    btn.click(
        fn=retrieve,
        inputs=[query, mode, topk, st_model, st_pool_embs, st_answer_pool],
        outputs=[out_tbl, out_why]
    )

# Avvio GUI (imposta share=True se vuoi link pubblico)
demo.launch(share=True)


Domande di prova per la GUI


1. *How can I cope with anxiety during exams?*
2. *I feel depressed most of the time, what should I do?*
3. *How do I build more confidence in social situations?*
4. *I’m having trouble sleeping because of stress, any advice?*
5. *How can I overcome a traumatic experience?*
6. *What are healthy ways to deal with anger?*
7. *How do I stop overthinking small problems?*
8. *I feel lonely even when surrounded by people, why?*

---

Spiegazione della GUI

* Con la **GUI Gradio** non stiamo più esplorando manualmente input/output del dataset.
* La GUI lavora solo sulle **risposte del dataset (output)**: queste sono state già pulite (*light* o *lemma*), deduplicate e trasformate in embeddings in anticipo.
* Quando scrivi una **nuova domanda** nella GUI:

  1. Viene pre-processata in base alla modalità scelta.
  2. Viene trasformata rapidamente in embedding
  3. Questo embedding viene confrontato con quelli delle risposte già calcolati (pool).
  4. La GUI ti mostra le Top-k risposte più simili con i loro punteggi di similarità coseno.

In pratica: **gli output sono fissi e precalcolati**, mentre la domanda che inserisci passa per un processo veloce di embedding al momento della query.

